In [29]:
from pyspark.sql import SparkSession

In [30]:
spark = SparkSession.builder \
    .appName("HELK Reader") \
    .master("spark://helk-spark-master:7077") \
    .enableHiveSupport() \
    .getOrCreate()

In [31]:
es_reader = (spark.read
    .format("org.elasticsearch.spark.sql")
    .option("inferSchema", "true")
    .option("es.read.field.as.array.include", "tags")
    .option("es.nodes","helk-elasticsearch:9200")
    .option("es.net.http.auth.user","elastic")
)

In [32]:
security_df = es_reader.load("logs-endpoint-winevent-security-*/")

In [33]:
security_df.createOrReplaceTempView("security_events")

In [34]:
sysmon_df = es_reader.load("logs-endpoint-winevent-sysmon-*/")

In [35]:
sysmon_df.createOrReplaceTempView("sysmon_events")

In [43]:
hook_and_hollowing = spark.sql(
'''
SELECT 
    c.host_name, 
    c.user_name,
    c.process_parent_name,
    d.process_target_name,
    c.user_logon_id AS Sysmon_EID_1_LogonId,
    d.process_granted_access
FROM security_events b
JOIN sysmon_events c
    ON c.process_name = b.process_name
    AND c.event_id = 1
    AND c.user_name = b.user_name
JOIN sysmon_events d
    ON c.process_guid = d.process_target_guid
    AND d.event_id = 10
    AND d.process_granted_access = "2097151" -- Assuming they are asking for full privileges. Was hard to narrow down the minimum rights needed to perform.
    AND c.process_parent_guid = d.process_guid
    AND c.process_name = d.process_target_name
    AND (NOT (c.process_parent_name = "msbuild.exe" AND d.process_target_name = "cmd.exe")) -- Blind spot + Assumption
JOIN sysmon_events e
    ON e.process_guid = c.process_parent_guid
    AND e.event_id = 5
    AND e.process_name = d.process_name
    AND c.process_parent_name = e.process_name 
JOIN sysmon_events f
    ON f.process_guid = c.process_parent_guid
    AND f.event_id = 7
    AND f.process_path = c.process_parent_path
    AND f.process_path = f.module_loaded
     AND f.signed = "false" -- Trivial for adversary to bypass. Here for extra context. 
WHERE 
    b.event_id = 4688
'''
).show(10,False)

+------------------+---------+--------------------+-------------------+--------------------+----------------------+
|host_name         |user_name|process_parent_name |process_target_name|Sysmon_EID_1_LogonId|process_granted_access|
+------------------+---------+--------------------+-------------------+--------------------+----------------------+
|win10.marvel.local|thor     |processhollowing.exe|cmd.exe            |9340634             |2097151               |
|win10.marvel.local|thor     |processhollowing.exe|svchost.exe        |9340634             |2097151               |
|win10.marvel.local|thor     |injectproc.exe      |cmd.exe            |537356              |2097151               |
+------------------+---------+--------------------+-------------------+--------------------+----------------------+



In [44]:
Hook_Injection = spark.sql(
'''
SELECT 
    b.host_name,
    b.process_parent_name,
    a.process_target_name,
    a.process_granted_access,
    a.user_reporter_name
FROM sysmon_events b
JOIN sysmon_events a
    ON a.process_target_guid = b.process_guid
    AND a.event_id = 10
    AND a.process_granted_access = "2097151" -- Assuming they are asking for full privileges. Was hard to narrow down the minimum rights needed to perform. API's were not specefic. 
    AND b.process_parent_name = a.process_name
    AND a.process_name != b.process_name
JOIN sysmon_events c
    ON b.process_guid = c.process_guid
    AND c.event_id = 13
    AND c.process_guid = a.process_target_guid
    AND c.process_name = a.process_target_name 
    AND (substring_index(c.registry_key_path, '\\\\', -1)) = b.process_name
JOIN sysmon_events d
    ON d.process_guid = a.process_target_guid
    AND d.event_id = 7
    AND d.module_loaded = a.process_target_path
    AND LOWER(d.OriginalFileName) = (substring_index(c.registry_key_path, '\\\\', -1))
    AND d.OriginalFileName = b.OriginalFileName
WHERE
     b.event_id = 1
    AND b.process_name = LOWER(b.OriginalFileName)
    AND (NOT (b.process_name = "rundll32.exe" AND b.process_parent_name = "svchost.exe")) -- Blind spot + Assumption
    AND (NOT (b.process_name = "notepad.exe" AND b.process_parent_name = "explorer.exe")) -- Blind spot + Assumption
    AND (NOT (b.process_name = "devenv.exe" AND b.process_parent_name = "explorer.exe")) -- Blind spot + Assumption
    AND (NOT (b.process_name = "cmd.exe" AND b.process_parent_name = "explorer.exe")) -- Blind spot + Assumption
'''
).show()

+------------------+--------------------+-------------------+----------------------+------------------+
|         host_name| process_parent_name|process_target_name|process_granted_access|user_reporter_name|
+------------------+--------------------+-------------------+----------------------+------------------+
|win10.marvel.local|      injectproc.exe|            cmd.exe|               2097151|            SYSTEM|
|win10.marvel.local|      injectproc.exe|        notepad.exe|               2097151|            SYSTEM|
|win10.marvel.local|processhollowing.exe|        svchost.exe|               2097151|            SYSTEM|
|win10.marvel.local|      injectproc.exe|        notepad.exe|               2097151|            SYSTEM|
|win10.marvel.local|             cmd.exe|     powershell.exe|               2097151|            SYSTEM|
|win10.marvel.local|             cmd.exe|     powershell.exe|               2097151|            SYSTEM|
+------------------+--------------------+-------------------+---

In [ ]:
# https://github.com/secrary/InjectProc